In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [3]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
X = torch.rand(2, 20)
net(X).shape

torch.Size([2, 10])

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)
    def forward(self, X):
        for module in self.children():
            X = module(X)
        return X


In [5]:
net = MySequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
net(X).shape

torch.Size([2, 10])

In [6]:
class MLP(nn.Module):
    def __init__(self):
        # Call the constructor of the parent class nn.Module to perform
        # the necessary initialization
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.out = nn.LazyLinear(10)
    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input X
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [7]:
net = MLP()
net(X).shape

torch.Size([2, 10])

In [8]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20))
        self.linear = nn.LazyLinear(20)
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(X @ self.rand_weight + 1)
        # Reuse the fully connected layer. This is equivalent to sharing
        # parameters with two fully connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [9]:
net = FixedHiddenMLP()
net(X)

tensor(0.1485, grad_fn=<SumBackward0>)

In [21]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(64), nn.ReLU(),
                                 nn.LazyLinear(32), nn.ReLU())
        self.linear = nn.LazyLinear(16)
    def forward(self, X):
        return self.linear(self.net(X))
chimera = nn.Sequential(NestMLP(), nn.LazyLinear(20), FixedHiddenMLP())
chimera(X)

tensor(0.1575, grad_fn=<SumBackward0>)

In [12]:
import torch
from torch import nn

In [13]:
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))
X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

In [14]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[ 0.2443, -0.0788,  0.2249,  0.0016,  0.3423, -0.0385, -0.2983,  0.1934]])),
             ('bias', tensor([0.3108]))])

In [15]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([0.3108]))

In [16]:
net[2].weight.grad == None

True

In [17]:
[(name, param.shape) for name, param in net.named_parameters()]


[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

In [18]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))
net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [19]:
import torch
from torch import nn

In [20]:
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(), nn.LazyLinear(1))
X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

In [22]:
def init_normal(module):
    if type(module) == nn.Linear:
        nn.init.normal_(module.weight, mean=0, std=0.01)
        nn.init.zeros_(module.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]


(tensor([ 0.0195, -0.0056, -0.0023,  0.0003]), tensor(0.))

In [23]:
def init_constant(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 1)
        nn.init.zeros_(module.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [24]:
def init_xavier(module):
    if type(module) == nn.Linear:
        nn.init.xavier_uniform_(module.weight)
def init_42(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 42)
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.5328, -0.1799, -0.3761, -0.0848])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [25]:
def my_init(module):
    if type(module) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in module.named_parameters()][0])
        nn.init.uniform_(module.weight, -10, 10)
        module.weight.data *= module.weight.data.abs() >= 5
net.apply(my_init)
net[0].weight[:2]


Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000, -5.6659,  8.4937,  0.0000],
        [ 0.0000, -8.1379,  0.0000,  0.0000]], grad_fn=<SliceBackward0>)

In [26]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -4.6659,  9.4937,  1.0000])

# 6.4 Code

In [27]:
import torch
from torch import nn
from d2l import torch as d2l

net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

In [29]:
net[0].weight

<UninitializedParameter>

# 6.5 Code

In [30]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [32]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, X):
        return X - X.mean()

In [33]:
layer = CenteredLayer()
layer(torch.tensor([1.0, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [34]:
net = nn.Sequential(nn.LazyLinear(128), CenteredLayer())

In [35]:
 Y = net(torch.rand(4, 8))
Y.mean()

tensor(-9.3132e-10, grad_fn=<MeanBackward0>)

In [36]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [37]:
linear = MyLinear(5, 3)
linear.weight


Parameter containing:
tensor([[-1.5910,  0.9220,  1.1167],
        [-0.7620, -1.9706, -0.1411],
        [ 0.3904, -0.3843, -0.5454],
        [ 0.2658, -0.5277,  0.3181],
        [ 1.1247, -1.5189, -0.4789]], requires_grad=True)

In [38]:
linear(torch.rand(2, 5))

tensor([[0.0606, 0.0000, 0.5750],
        [0.0000, 0.0000, 0.5314]])

In [40]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

# 6.6 Code

In [41]:
import torch
from torch import nn
from torch.nn import functional as F

In [42]:
x = torch.arange(4)
torch.save(x, 'x-file')

In [43]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [44]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [45]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [46]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.output = nn.LazyLinear(10)
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))
net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [47]:
torch.save(net.state_dict(), 'mlp.params')

In [49]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): LazyLinear(in_features=0, out_features=256, bias=True)
  (output): LazyLinear(in_features=0, out_features=10, bias=True)
)

In [50]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

# 6.7 Code

In [64]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [65]:
import torch
from torch import nn
from d2l import torch as d2l


I don't have an Nvidia gpu

In [66]:
def cpu():  #@save
    """Get the CPU device."""
    return torch.device('cpu')
def gpu(i=0):  #@save
    """Get a GPU device."""
    return torch.device(f'cuda:{i}')
cpu(), gpu(), gpu(1)

(device(type='cpu'),
 device(type='cuda', index=0),
 device(type='cuda', index=1))

In [67]:
def num_gpus():  #@save
    """Get the number of available GPUs."""
    return torch.cuda.device_count()
num_gpus()

0

In [68]:
def try_gpu(i=0):  #@save
    """Return gpu(i) if exists, otherwise return cpu()."""
    if num_gpus() >= i + 1:
        return gpu(i)
    return cpu()
def try_all_gpus():  #@save
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    return [gpu(i) for i in range(num_gpus())]
try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cpu'), device(type='cpu'), [])

In [69]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [70]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [71]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.1540, 0.1914, 0.4815],
        [0.4674, 0.6985, 0.2068]])

In [72]:
Z = X.cuda(1)
print(X)
print(Z)

AssertionError: Torch not compiled with CUDA enabled

In [73]:
 net = nn.Sequential(nn.LazyLinear(1))
net = net.to(device=try_gpu())

In [74]:
net(X)

tensor([[0.7807],
        [0.7807]], grad_fn=<AddmmBackward0>)

In [75]:
net[0].weight.data.device

device(type='cpu')

In [77]:
@d2l.add_to_class(d2l.Trainer)  #@save
def __init__(self, max_epochs, num_gpus=0, gradient_clip_val=0):
    self.save_hyperparameters()
    self.gpus = [d2l.gpu(i) for i in range(min(num_gpus, d2l.num_gpus()))]
@d2l.add_to_class(d2l.Trainer)  #@save
def prepare_batch(self, batch):
    if self.gpus:
        batch = [a.to(self.gpus[0]) for a in batch]
    return batch
@d2l.add_to_class(d2l.Trainer)  #@save
def prepare_model(self, model):
    model.trainer = self
    model.board.xlim = [0, self.max_epochs]
    if self.gpus:
        model.to(self.gpus[0])
    self.model = model